In [2]:
from cergen import gergen, rastgele_dogal, rastgele_gercek, Operation
from typing import Literal, Optional, Union

import math
import pandas as pd

In [ ]:
"""
TYPE ALIASES & CONSTANTS
"""

ActivationType = Literal['softmax', 'relu']

In [ ]:
def ReLU(x: Union[float, int]) -> Union[float, int]:
    return max(0, x)

def softmax(x: list[Union[float, int]], i: int) -> float:
    return math.exp(x[i]) / sum(math.exp(xi) for xi in x)

In [ ]:
class ForwardPass(Operation):
    def ileri(self, x: gergen, W: gergen, b: gergen) -> gergen:
        """
        The forward pass of a layer, which computes the output of the layer given the input x, weights W, and biases b.
        """
        return x.ic_carpim(W) + b


class Katman:
    x: gergen = None

    W: gergen = None

    b: gergen = None

    outputs: gergen = None

    activation: str = None
    
    forward_pass = ForwardPass()

    def __init__(self, input_size: int, output_size: int, activation: Optional[ActivationType] = None):
        """
        With the given input and output sizes, the layer’s weighs and biases are initialised with appropriate shapes using the rastgele_gercek() function.
        You can choose the mean and the
        standard deviation of the initial parameters as you wish, e.g., using Xavier or He initialization
        methods.
        """
        
        self.x = rastgele_dogal((input_size, 1))
        self.W = rastgele_gercek((output_size, input_size))
        self.b = rastgele_gercek((output_size, 1))
        self.activation = activation

        raw_forward_pass: gergen = self.forward_pass(self.x, self.W, self.b)

        if self.activation == 'relu':
            self.outputs = raw_forward_pass.map(ReLU)

        elif self.activation == 'softmax':
            self.outputs = raw_forward_pass.map(softmax)

        else:
            self.outputs = raw_forward_pass
